In [1]:
import pandas as pd
import numpy as np

In [2]:
#User features 
user_features = pd.read_csv("csvfiles/user_features_train.csv")


In [3]:
#Users meta data 
users = pd.read_csv("csvfiles/users_train.csv")

In [4]:
#Target_Train 
target = pd.read_csv("csvfiles/targets_train.csv")

In [5]:
user_features.head()

,ID,first_prediction,RetentionD0,RetentionD1,RetentionD2,RetentionD3,RetentionD4,RetentionD5,RetentionD6,RetentionD7,...,IAPRevenueD6,IAPRevenueD7,IAPRevenueD8,IAPRevenueD9,IAPRevenueD10,IAPRevenueD11,IAPRevenueD12,IAPRevenueD13,IAPRevenueD14,IAPRevenueD15
0,0,3.314099,True,False,False,False,False,False,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,1.681524,True,False,False,False,False,False,True,True,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,10.718750,True,False,False,False,False,False,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,5.100000,True,True,True,False,False,False,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,2.091409,True,False,False,False,False,False,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
users.head()

,ID,first_open_date,first_open_timestamp,local_first_open_timestamp,country,platform,device_category,device_brand,device_model,has_ios_att_permission,ad_network
0,0,2024-03-02,1709355895042000,1709334295042000,Mexico,Android,mobile,Xiaomi,Redmi A2,False,unityads_int
1,1,2024-03-19,1710824539731000,1710806539731000,Peru,Android,mobile,Samsung,Galaxy A13,False,applovin_int
2,2,2024-03-18,1710731043082000,1710720243082000,Brazil,Android,mobile,Xiaomi,Redmi 12,False,applovin_int
3,3,2024-03-03,1709455862260000,1709441462260000,Dominican Republic,iOS,mobile,Apple,iPhone 11 Pro Max,False,NaN
4,4,2024-04-30,1714482477190000,1714464477190000,Ecuador,Android,mobile,Motorola,Moto E22,False,applovin_int


In [7]:
target.head()

,ID,TARGET
0,0,0.000000
1,1,0.018892
2,2,0.000000
3,3,0.046650
4,4,0.014680


In [8]:
user_features.isna().sum()

ID                      0
first_prediction    25735
RetentionD0             0
RetentionD1             0
RetentionD2             0
                    ...  
IAPRevenueD11           0
IAPRevenueD12           0
IAPRevenueD13           0
IAPRevenueD14           0
IAPRevenueD15           0
Length: 76, dtype: int64

In [9]:
target.isna().sum()

ID        0
TARGET    0
dtype: int64

In [10]:
users.isna().sum()

ID                                 0
first_open_date                    0
first_open_timestamp               0
local_first_open_timestamp         0
country                           82
platform                           0
device_category                    0
device_brand                    5840
device_model                       0
has_ios_att_permission             0
ad_network                    310470
dtype: int64

In [11]:
users["has_ios_att_permission"].value_counts()

has_ios_att_permission
False    764921
True     113673
Name: count, dtype: int64

In [12]:
users["device_brand"].value_counts()

device_brand
Apple          363623
Samsung        225831
Xiaomi          82158
Motorola        76009
OPPO            22920
                ...  
Aiprotablet         1
Fossibot            1
Meitu               1
Tagital             1
Movisun             1
Name: count, Length: 295, dtype: int64

In [13]:
users["ID"]

0              0
1              1
2              2
3              3
4              4
           ...  
878589    878589
878590    878590
878591    878591
878592    878592
878593    878593
Name: ID, Length: 878594, dtype: int64

In [14]:
merged_df = pd.merge(users, user_features, on = "ID")
df = pd.merge(merged_df, target, on="ID")

In [15]:
#Leaving outside the players that uninstalled the game

retention_columns = [f"RetentionD{i}" for i in range(0, 16)]

#A function to calculate last active day of player

def last_active_day(row):
    for day in range(15,-1,-1):
        if row[f"RetentionD{day}"] == True:
            return day
    return 0 


In [16]:
df["Last Active Day"] = df.apply(last_active_day, axis=1)

In [17]:
df["Last Active Day"].head(20)

0     12
1      9
2      0
3      2
4      0
5      2
6      2
7      0
8      6
9      1
10     0
11     1
12    15
13     7
14     2
15     8
16     0
17     0
18     0
19     9
Name: Last Active Day, dtype: int64

In [18]:
def count_retention_values(row):
    true_count = sum(row[col] == True for col in retention_columns)
    false_count = sum(row[col] == False for col in retention_columns)
    return pd.Series([true_count, false_count], index=['True_count', 'False_count'])

In [19]:
#8. günden sonra oynaması lazım 5 günden az oynamaması lazım

df[['True_count', 'False_count']] = df.apply(count_retention_values, axis=1)

In [20]:
def active_after_day8(row):
    return any(row[f"RetentionD{i}"] == True for i in range(8, 16))

In [21]:
def active_in_first_8_days(row):
    return sum(row[f"RetentionD{i}"] == True for i in range(0, 8)) >= 5

In [22]:
df['Active after 8'] = df.apply(active_after_day8, axis=1)
df['Active in first 8 days'] = df.apply(active_in_first_8_days, axis=1)

In [23]:
filtered_df = df[(df['Active after 8'] == True) | ((df["Active after 8"] == False) & (df['Active in first 8 days'] == True))]

In [24]:
excluded_df = df[~df['ID'].isin(filtered_df['ID'])]

In [33]:
df.head()

,ID,first_open_date,first_open_timestamp,local_first_open_timestamp,country,platform,device_category,device_brand,device_model,has_ios_att_permission,...,IAPRevenueD12,IAPRevenueD13,IAPRevenueD14,IAPRevenueD15,TARGET,Last Active Day,True_count,False_count,Active after 8,Active in first 8 days
0,0,2024-03-02,1709355895042000,1709334295042000,Mexico,Android,mobile,Xiaomi,Redmi A2,False,...,0.0,0.0,0.0,0.0,0.000000,12,2,14,True,False
1,1,2024-03-19,1710824539731000,1710806539731000,Peru,Android,mobile,Samsung,Galaxy A13,False,...,0.0,0.0,0.0,0.0,0.018892,9,4,12,True,False
2,2,2024-03-18,1710731043082000,1710720243082000,Brazil,Android,mobile,Xiaomi,Redmi 12,False,...,0.0,0.0,0.0,0.0,0.000000,0,1,15,False,False
3,3,2024-03-03,1709455862260000,1709441462260000,Dominican Republic,iOS,mobile,Apple,iPhone 11 Pro Max,False,...,0.0,0.0,0.0,0.0,0.046650,2,3,13,False,False
4,4,2024-04-30,1714482477190000,1714464477190000,Ecuador,Android,mobile,Motorola,Moto E22,False,...,0.0,0.0,0.0,0.0,0.014680,0,1,15,False,False


In [30]:
excluded_df.head()

,ID,first_open_date,first_open_timestamp,local_first_open_timestamp,country,platform,device_category,device_brand,device_model,has_ios_att_permission,...,IAPRevenueD13,IAPRevenueD14,IAPRevenueD15,TARGET,Last Active Day,True_count,False_count,Active after 8,Active in first 8 days,15_day_revenue
2,2,2024-03-18,1710731043082000,1710720243082000,Brazil,Android,mobile,Xiaomi,Redmi 12,False,...,0.0,0.0,0.0,0.00000,0,1,15,False,False,0.0000
3,3,2024-03-03,1709455862260000,1709441462260000,Dominican Republic,iOS,mobile,Apple,iPhone 11 Pro Max,False,...,0.0,0.0,0.0,0.04665,2,3,13,False,False,0.0445
4,4,2024-04-30,1714482477190000,1714464477190000,Ecuador,Android,mobile,Motorola,Moto E22,False,...,0.0,0.0,0.0,0.01468,0,1,15,False,False,0.0000
5,5,2024-04-23,1713905887691000,1713887887691000,Colombia,Android,mobile,Xiaomi,Redmi Note 8,False,...,0.0,0.0,0.0,0.00000,2,3,13,False,False,0.0000
6,6,2024-03-30,1711829156974000,1711814756974000,Venezuela,Android,mobile,Samsung,Galaxy S20 FE,False,...,0.0,0.0,0.0,0.00000,2,2,14,False,False,0.0000


In [32]:
filtered_df.head()

,ID,first_open_date,first_open_timestamp,local_first_open_timestamp,country,platform,device_category,device_brand,device_model,has_ios_att_permission,...,IAPRevenueD12,IAPRevenueD13,IAPRevenueD14,IAPRevenueD15,TARGET,Last Active Day,True_count,False_count,Active after 8,Active in first 8 days
0,0,2024-03-02,1709355895042000,1709334295042000,Mexico,Android,mobile,Xiaomi,Redmi A2,False,...,0.0,0.0,0.0,0.0,0.000000,12,2,14,True,False
1,1,2024-03-19,1710824539731000,1710806539731000,Peru,Android,mobile,Samsung,Galaxy A13,False,...,0.0,0.0,0.0,0.0,0.018892,9,4,12,True,False
12,12,2024-05-04,1714783839460000,1714791039460000,Germany,iOS,mobile,Apple,iPhone 14 Pro Max,False,...,0.0,0.0,0.0,0.0,1.265949,15,7,9,True,False
15,15,2024-02-05,1707146971878000,1707166771878000,India,Android,mobile,Xiaomi,Redmi 12 5G,False,...,0.0,0.0,0.0,0.0,0.044076,8,2,14,True,False
19,19,2024-01-06,1704551875588000,1704541075588000,Brazil,Android,mobile,Motorola,Moto G84 5G,False,...,0.0,0.0,0.0,0.0,0.046317,9,4,12,True,False


In [25]:
def calculate_15_day_revenue(row):
    ad_revenue = sum(row[f"AdRevenueD{i}"] for i in range(1, 16) if row[f"AdRevenueD{i}"] is not None)
    iap_revenue = sum(row[f"IAPRevenueD{i}"] for i in range(1, 16) if row[f"IAPRevenueD{i}"] is not None)
    return ad_revenue + iap_revenue

excluded_df['15_day_revenue'] = excluded_df.apply(calculate_15_day_revenue, axis=1)

C:\Users\omerf\AppData\Local\Temp\ipykernel_1140\3215362121.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  excluded_df['15_day_revenue'] = excluded_df.apply(calculate_15_day_revenue, axis=1)


In [35]:
excluded_df["15_day_revenue"].head()

2    0.0000
3    0.0445
4    0.0000
5    0.0000
6    0.0000
Name: 15_day_revenue, dtype: float64

In [39]:
filtered_df.drop(columns=['Last Active Day',
       'True_count', 'False_count', 'Active after 8',
       'Active in first 8 days'], inplace=True)

C:\Users\omerf\AppData\Local\Temp\ipykernel_1140\3409789487.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.drop(columns=['Last Active Day',


In [40]:
filtered_df.columns

Index(['ID', 'first_open_date', 'first_open_timestamp',
       'local_first_open_timestamp', 'country', 'platform', 'device_category',
       'device_brand', 'device_model', 'has_ios_att_permission', 'ad_network',
       'first_prediction', 'RetentionD0', 'RetentionD1', 'RetentionD2',
       'RetentionD3', 'RetentionD4', 'RetentionD5', 'RetentionD6',
       'RetentionD7', 'RetentionD8', 'RetentionD9', 'RetentionD10',
       'RetentionD11', 'RetentionD12', 'RetentionD13', 'RetentionD14',
       'RetentionD15', 'LevelAdvancedCountD0', 'LevelAdvancedCountD1',
       'LevelAdvancedCountD2', 'LevelAdvancedCountD3', 'LevelAdvancedCountD4',
       'LevelAdvancedCountD5', 'LevelAdvancedCountD6', 'LevelAdvancedCountD7',
       'LevelAdvancedCountD8', 'LevelAdvancedCountD9', 'LevelAdvancedCountD10',
       'LevelAdvancedCountD11', 'LevelAdvancedCountD12',
       'LevelAdvancedCountD13', 'LevelAdvancedCountD14',
       'LevelAdvancedCountD15', 'Level_1_Duration', 'Level_2_Duration',
       'Level

In [42]:
users.isna().sum()

ID                                 0
first_open_date                    0
first_open_timestamp               0
local_first_open_timestamp         0
country                           82
platform                           0
device_category                    0
device_brand                    5840
device_model                       0
has_ios_att_permission             0
ad_network                    310470
dtype: int64

In [43]:
user_features.isna().sum()

ID                      0
first_prediction    25735
RetentionD0             0
RetentionD1             0
RetentionD2             0
                    ...  
IAPRevenueD11           0
IAPRevenueD12           0
IAPRevenueD13           0
IAPRevenueD14           0
IAPRevenueD15           0
Length: 76, dtype: int64

In [44]:
filtered_df["first_prediction"].isna().sum()

4876